In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import multiprocessing as mp
import pickle

from scipy.spatial.distance import pdist, squareform, cdist
from scipy.sparse import csr_matrix, vstack
from scipy.sparse.csgraph import dijkstra, connected_components, bellman_ford, maximum_flow, shortest_path
from itertools import chain, combinations, product

import os

from reeb_graph import Reeb_Graph
from reeb_aux import *

In [2]:
"""
Setup Data Folder
"""

data_folder = './Data'
results_folder = './Results'
compositions = ['/67-33','/70-30','/75-25']
#versions = ['/Quench_1','/Quench_2','/Quench_3','/Quench_4','/Quench_5','/Quench_6']
versions = ['/Quench_4','/Quench_5','/Quench_6']
#add_folder = '/MQ'
add_folder = '/MSD'
#temperature = ['/300','/400','/500','/600','/700']
temperature = ['']

data_set = '/md.statsis3'
axes_names = ['_x_','_y_','_z_']
MP = False

T = temperature[0]
results_folder = './Results'+T


# Functions 

In [3]:
from numba import jit, int32, prange, generated_jit

@jit(nopython=True, parallel=True, fastmath=True)
def MSD(A,p=1):
    
    tmp = len(A)
    size = len(A[0,:,0])
    axes_aux = np.array([0,1,2])
    RES = np.zeros((size,tmp-1))
    
    for t in range(tmp-1):
        for i in range(size):
            v = A[0,i,:].reshape(1,-1)
            w = A[t+1,i,:].reshape(1,-1)            
            RES[i,t] = numba_dist(v,w)[0][0]**p 
            
    return RES
        
@jit(nopython=True, parallel=True, fastmath=True)
def MSD_mask(A,p=1,mask=np.array([0,0,1])):
    
    tmp = len(A)
    size = len(A[0,:,0])
    axes_aux = np.array([0,1,2])
    RES = np.zeros((size,tmp-1))
            
    for t in range(tmp-1):
        for i in range(size):
            v = A[0,i,:].reshape(1,-1)*mask
            w = A[t+1,i,:].reshape(1,-1)*mask
            
            RES[i,t] = numba_dist(v,w)[0][0]**p 
            
    return RES
    
    
    
def aux_MSD(Li,p=1,mask=None):
    
    if mask is None:        
        d_L = MSD(Li,p)
    else:
        d_L = MSD_mask(Li,p,mask)
        
    return d_L

In [4]:
import networkx as nx
import pyomo.environ as pyo
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)


def circular_max_flow(reeb):
    
    edges = reeb.E+1
    
    pts_bottom = np.unique(reeb.REEB_GRAPH[:,0])
    pts_bottom = pts_bottom[pts_bottom>-1]
    pts_top = np.unique(reeb.REEB_GRAPH[:,-1])
    pts_top = pts_top[pts_top>-1]
    
    add_edges_bottom = np.vstack([np.zeros_like(pts_bottom),pts_bottom + 1]).T
    add_edges_top = np.vstack([pts_top + 1, np.zeros_like(pts_top) + np.max(reeb.G.nodes) + 2]).T
    
    constr_edges = np.zeros((len(add_edges_bottom),4))
#    constr_edges = np.hstack([add_edges_bottom, add_edges_top])
    constr_edges[:,:2] = add_edges_bottom
    constr_edges[:,2:] = add_edges_top
    
    edges = np.vstack([add_edges_bottom,edges,add_edges_top])
    edges_to_idx = {}

    for e in edges:
        edges_to_idx[tuple(e)] = np.argmin(np.linalg.norm(edges-e,axis=-1)) 
        
    N = np.sum(reeb.D_reeb_w)
    cap = {}
    
    for e in reeb.E:
        cap[tuple(e+1)] = reeb.G.edges[tuple(e)]['capacity']
        
    for e in add_edges_bottom:
        cap[tuple(e)] = N
    
    for e in add_edges_top:
        cap[tuple(e)] = N
    
    cost = pyo.ConcreteModel()
    cost.n = len(edges) 

    cost.f = pyo.Var(np.arange(cost.n),
                     domain=pyo.NonNegativeReals,initialize=0)
    
    flow = sum([cost.f[edges_to_idx[(0,v+1)]] for v in pts_bottom]) 

    cost.obj=pyo.Objective(rule=flow, sense=pyo.maximize)
    cost.costr=pyo.ConstraintList()

    for e in edges:
        cost.costr.add(cost.f[edges_to_idx[tuple(e)]] <= cap[tuple(e)])

    for e in constr_edges:
        cost.costr.add(cost.f[edges_to_idx[tuple(e[:2])]] == cost.f[edges_to_idx[tuple(e[2:])]])


    for v in list(reeb.G.nodes):
        
        neigh = list(reeb.G[v])
        
        if v in pts_bottom:
            neigh = neigh+[-1]  
        elif v in pts_top:
            neigh = neigh+[np.max(reeb.G.nodes) + 1]  
            
        if len(neigh)>0:
            flx_in = sum([cost.f[edges_to_idx[(u+1,v+1)]] for u in neigh if u<v])
            flx_out = sum([cost.f[edges_to_idx[(v+1,u+1)]] for u in neigh if u>v])
            cost.costr.add(flx_in-flx_out==0)

    solver = pyo.SolverFactory('gurobi')
    S=solver.solve(cost)

    flx = cost.obj()

    f_eval = np.zeros_like(edges[:,0])

    for key,val in cost.f.extract_values().items():
                f_eval[key]=val 

    flow_eval = np.sum([f_eval[edges_to_idx[(0,v+1)]] for v in pts_bottom]) 
    
    return flow_eval

# Flow

In [5]:
n = 2

In [8]:
radius = [-1,0,1]

FLOWS = np.zeros((len(compositions),len(versions)*len(axes_names),len(radius)))
MSD_ = np.zeros((len(compositions),len(versions)*len(axes_names)))
RADII = np.zeros((len(compositions),len(versions)*len(axes_names)))

for c,comp in enumerate(compositions): 
    
    print('Doing Glass ', comp,'                 ')
    
    var_by_comp = []
    r_by_comp = []
    
    flow = np.array([0,0,0])
    
    cnt = 0
    
    for vers in versions:

        print('Doing Version ', vers,'                    ')
        

        for axis,V in enumerate(canonical_matrix_grid()):
                       
            for k,r in enumerate(radius):

                
                name = results_folder + comp + '_' + vers[1:] + '_reeb' + axes_names[axis] + str(r) + '.pickle' 
                zip_name = comp[1:] + '_' + vers[1:] + '_reeb' + axes_names[axis] + str(r) + '.zip'      
                name_ = comp[1:] + '_' + vers[1:] + '_reeb' + axes_names[axis] + str(r) + '.pickle' 
                 

                cmd = 'cd ' + results_folder + ' && unzip '+ zip_name + ' ' + name_
                os.system(cmd)

                with open(name, "rb") as fp:
                    reeb = pickle.load(fp)
                    
                """Flow"""

                flow = circular_max_flow(reeb)*(reeb.unit_2d)
                FLOWS[c,cnt,k] = flow
                
                if k==0:
                    
                    """MSD"""
                    
                    disp = aux_MSD(reeb.Li,2,V[:,-1])[:,-1]

                    msd = np.mean(disp)
                    MSD_[c,cnt] = msd
                    
                    RADII[c,cnt] = np.mean(reeb.balls_radii)
                
                cmd = 'rm '+ name 
                os.system(cmd)
                
                print(comp,vers,axes_names[axis],', Radius ',r,'                     ')
                    
            cnt+= 1

Doing Glass  /67-33                  
Doing Version  /Quench_4                     
Archive:  67-33_Quench_4_reeb_x_-1.zip
  inflating: 67-33_Quench_4_reeb_x_-1.pickle  
/67-33 /Quench_4 _x_ , Radius  -1                      
Archive:  67-33_Quench_4_reeb_x_0.zip
  inflating: 67-33_Quench_4_reeb_x_0.pickle  
/67-33 /Quench_4 _x_ , Radius  0                      
Archive:  67-33_Quench_4_reeb_x_1.zip
  inflating: 67-33_Quench_4_reeb_x_1.pickle  
/67-33 /Quench_4 _x_ , Radius  1                      
Archive:  67-33_Quench_4_reeb_y_-1.zip
  inflating: 67-33_Quench_4_reeb_y_-1.pickle  
/67-33 /Quench_4 _y_ , Radius  -1                      
Archive:  67-33_Quench_4_reeb_y_0.zip
  inflating: 67-33_Quench_4_reeb_y_0.pickle  
/67-33 /Quench_4 _y_ , Radius  0                      
Archive:  67-33_Quench_4_reeb_y_1.zip
  inflating: 67-33_Quench_4_reeb_y_1.pickle  
/67-33 /Quench_4 _y_ , Radius  1                      
Archive:  67-33_Quench_4_reeb_z_-1.zip
  inflating: 67-33_Quench_4_reeb_z_-1

  inflating: 75-25_Quench_4_reeb_x_0.pickle  
/75-25 /Quench_4 _x_ , Radius  0                      
Archive:  75-25_Quench_4_reeb_x_1.zip
  inflating: 75-25_Quench_4_reeb_x_1.pickle  
/75-25 /Quench_4 _x_ , Radius  1                      
Archive:  75-25_Quench_4_reeb_y_-1.zip
  inflating: 75-25_Quench_4_reeb_y_-1.pickle  
/75-25 /Quench_4 _y_ , Radius  -1                      
Archive:  75-25_Quench_4_reeb_y_0.zip
  inflating: 75-25_Quench_4_reeb_y_0.pickle  
/75-25 /Quench_4 _y_ , Radius  0                      
Archive:  75-25_Quench_4_reeb_y_1.zip
  inflating: 75-25_Quench_4_reeb_y_1.pickle  
/75-25 /Quench_4 _y_ , Radius  1                      
Archive:  75-25_Quench_4_reeb_z_-1.zip
  inflating: 75-25_Quench_4_reeb_z_-1.pickle  
/75-25 /Quench_4 _z_ , Radius  -1                      
Archive:  75-25_Quench_4_reeb_z_0.zip
  inflating: 75-25_Quench_4_reeb_z_0.pickle  
/75-25 /Quench_4 _z_ , Radius  0                      
Archive:  75-25_Quench_4_reeb_z_1.zip
  inflating: 75-25_Qu

In [10]:
np.save('FLOW_li2s_300',FLOWS)
np.save('MSD_li2s_300',MSD_)
np.save('RADII_li2s_300',RADII)

In [9]:
print(FLOWS.shape,MSD_.shape)

(3, 9, 3) (3, 9)
